In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
def print_results(y_true, y_pred):
    print(confusion_matrix(y_true, y_pred))
    print('F1-score:', f1_score(y_true, y_pred, average='macro'))
def plot_validation_curve(model_grid, param_name, params=None):
    # Рисуем валидационную кривую
    # По оси х --- значения гиперпараметров (param_***)
    # По оси y --- значения метрики (mean_test_score)

    results_df = pd.DataFrame(model_grid.cv_results_)
    
    if params == None:
        plt.plot(results_df['param_'+param_name], results_df['mean_test_score'])
    else:
        plt.plot(params, results_df['mean_test_score'])

    # Подписываем оси и график
    plt.xlabel(param_name)
    plt.ylabel('Test F1 score')
    plt.title('Validation curve')
    plt.show()

In [ ]:
churn = pd.read_csv('/kaggle/input/churn-in-telecoms-dataset/bigml_59c28831336c6604c800002a.csv')
churn.head()

In [ ]:
churn["international plan"] = churn["international plan"].map({"yes":1, "no": 0})
churn["voice mail plan"] = churn["voice mail plan"].map({"yes":1, "no":0})
churn['churn'] = churn['churn'].map({True:1, False:0})
df = churn.drop(["state", "phone number"], axis = 1)
df.head()

In [ ]:
df.dtypes

In [ ]:
numeric = [x for x in df.columns.values if df[x].dtype!='object']
len(numeric)

In [ ]:
df_numeric = df[numeric]
df_clean = df_numeric.iloc[:, :116].dropna()
from sklearn.model_selection import train_test_split
X = df_clean.drop('churn', axis=1)
y = df_clean['churn']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, 
                                                      random_state=12)
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(),
                            max_samples=0.5, max_features=1.0, n_estimators=10)

bagging.fit(X_train, y_train)

In [ ]:
y_pred = bagging.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_valid)
print_results(y_valid, y_pred_knn)

In [ ]:
bagging = BaggingClassifier(KNeighborsClassifier(),
                            max_samples=0.5, max_features=1.0, n_estimators=20)

bagging.fit(X_train, y_train)

In [ ]:
y_pred = bagging.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
rf = RandomForestClassifier(n_estimators=10, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
et = ExtraTreesClassifier(n_estimators=10, random_state=0, n_jobs=-1)
et.fit(X_train, y_train)
y_pred = et.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
et_params={'n_estimators': np.arange(10, 121, 10)} # словарь параметров (ключ: набор возможных значений)

et_grid = GridSearchCV(et, et_params, cv=5, scoring='f1_macro', n_jobs=-1) # кросс-валидация по 5 блокам
et_grid.fit(X_train, y_train)

In [ ]:
et_grid.best_score_

In [ ]:
plot_validation_curve(et_grid, 'n_estimators')

In [ ]:
ab = AdaBoostClassifier(n_estimators=100, random_state=21)
ab.fit(X_train, y_train)

In [ ]:
y_pred = ab.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
ab = AdaBoostClassifier(base_estimator=LogisticRegression(),
                        n_estimators=100, random_state=21)
ab.fit(X_train, y_train)

In [ ]:
y_pred = ab.predict(X_valid)
print_results(y_valid, y_pred)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
estimators = [('knn', KNeighborsClassifier()), 
              ('tree', DecisionTreeClassifier(max_depth=7, random_state=42)),
              ('log_reg', LogisticRegression())]
stack = StackingClassifier(estimators=estimators,
                           final_estimator=RandomForestClassifier(n_estimators=10))
stack.fit(X_train, y_train)

In [ ]:
y_pred = stack.predict(X_valid)
print_results(y_valid, y_pred)